In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 500)

# Import Wikipedia Page as a text file and then use Beautiful Soup LXML to parse the text file

In [ ]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

# Search through the file to locate only the table of data that we need and then extract the column headers

In [ ]:
extract_table = soup.find('table',{'class':'wikitable sortable'})
get_columns = extract_table.findAll('th')
titles = []
for c in get_columns:
    titles.append(c.get_text().strip('\n'))
titles

# Extract Data Row by Row

In [ ]:
entries = []

for r in extract_table.findAll('tr'):
    get_entry = r.findAll('td')
    entries.append([td.text.strip() for td in get_entry])

# Remove all rows where value of Bourough is equal to 'Not Assigned'

In [ ]:
na_index = []

df = pd.DataFrame(columns = titles, data = entries)
df = df.drop(0)
ndf = df[df.Borough != 'Not assigned']

# Join Rows together where Postcode and Bourough match. Join the Neighbourhood values together and seperate by comma.
# Then find the index value of all rows where a Borough is assigned but has a Neighbourhood value of 'Not Assigned'. Use this index value to locate all such rows and then reassign the Neighbourhood to match the value of the rows Borough

In [ ]:
ndf = ndf.reset_index(drop = True)
ndf = ndf.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()
na_index = ndf.loc[ndf['Neighbourhood'] == 'Not assigned'].index
print(na_index)
for idx in na_index:
    ndf.iloc[idx]['Neighbourhood'] = ndf.iloc[idx]['Borough']

# Print Shape

In [305]:
ndf.shape

(103, 3)

In [ ]:
ndf.loc[ndf['Postcode'] == 'M4B']